<a href="https://colab.research.google.com/github/DanielWill-1/DanielWill-1/blob/main/VAE(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers,Model
from tensorflow.keras import datasets
from tensorflow.keras.losses import MeanSquaredError, KLDivergence, binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from numpy import cov,trace,iscomplexobj, asarray
from scipy.linalg import sqrtm
from skimage.transform import resize

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [3]:
x_train = x_train.astype('float32')/ 255.0
x_test=x_test.astype('float32')/ 255.0

In [4]:
latent_dim= 1024
image_shape=x_train[0].shape
mse_loss=MeanSquaredError()
kl_loss=KLDivergence()

In [5]:
class VAE(keras.Model):
  def __init__(self,latent_dim, image_shape,beta):
    super(VAE,self).__init__()
    self.latent_dim=latent_dim
    self.image_shape=image_shape
    self.beta=beta

    self.encoder=tf.keras.Sequential([
        layers.Input(shape=image_shape, name="Encoder_Input_Layer"),
        layers.Conv2D(32,3,strides=2,activation="relu", padding='same', name="Encoder_Conv2D_2"),
        layers.Conv2D(64,3,strides=2,activation="relu", padding='same', name="Encoder_Conv2D_4"),
        layers.Conv2D(128,3,strides=2,activation="relu", padding='same', name="Encoder_Conv2D_6"),
        layers.Flatten(name="Encoder_Flatten"),

    ])

    self.z_mean=layers.Dense(latent_dim,name="z_mean")
    self.z_log_var=layers.Dense(latent_dim,name="z__log_var")

    self.decoder=tf.keras.Sequential([
        layers.InputLayer(input_shape=(latent_dim,)),
        layers.Dense(8*8*64, activation='relu'),
        layers.Reshape((8, 8, 64)),
        layers.Conv2DTranspose(64,kernel_size=3,strides=2,activation='relu',padding='same'),
        layers.Conv2DTranspose(32,kernel_size=3,strides=2,activation='relu',padding='same'),
        layers.Conv2DTranspose(3,kernel_size=3,strides=1,activation='sigmoid',padding='same'),
    ])

    self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
    self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
    self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

  def encode(self,data):
    x=self.encoder(data)
    z_mean,z_log_var= self.z_mean(x),self.z_log_var(x)
    return z_mean,z_log_var

  def reparameterize(self,z_mean,z_log_var):
    batch=tf.shape(z_mean)[0]
    dim=tf.shape(z_mean)[1]
    epsilon=tf.keras.backend.random_normal(shape=(batch,dim))
    z=z_mean+tf.exp(0.5*z_log_var)*epsilon
    return z

  def call(self,x):
    mean,log_var=self.encode(x)
    z=self.reparameterize(mean,log_var)
    x_recon=self.decode(z)
    return x_recon, mean, log_var

  def decode(self,data):
    return self.decoder(data)

  @property
  def metrics(self):
    return [
        self.total_loss_tracker,
        self.reconstruction_loss_tracker,
        self.kl_loss_tracker,
    ]

  def recon_loss(self,data,reconstruction):
    return tf.reduce_mean(binary_crossentropy(data, reconstruction))

  def kl_divergence(self,Z_log_var, Z_mu):
    kl=-0.5*tf.reduce_mean(1+Z_log_var-Z_mu**2-tf.math.exp(Z_log_var))
    return self.beta*kl

  def train_step(self,data):
    with tf.GradientTape() as tape:
      z_mean, z_log_var=self.encode(data)
      z=self.reparameterize(z_mean,z_log_var)
      reconstruction=self.decode(z)
      reconstruction_loss=self.recon_loss(data,reconstruction)
      kl_loss=self.kl_divergence(z_log_var, z_mean)
      trial_loss=reconstruction_loss+kl_loss
    grads=tape.gradient(trial_loss,self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads,self.trainable_weights))
    self.total_loss_tracker.update_state(trial_loss)
    self.reconstruction_loss_tracker.update_state(reconstruction_loss)
    self.kl_loss_tracker.update_state(kl_loss)
    return{
        "loss":self.total_loss_tracker.result(),
        "reconstruction_loss":self.reconstruction_loss_tracker.result(),
        "kl_loss":self.kl_loss_tracker.result(),
    }


In [6]:
vae=VAE(latent_dim,image_shape,0.3)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
vae.compile(optimizer=keras.optimizers.Adam())
early_stopping=EarlyStopping(monitor='loss',patience=10,verbose=1)
history=vae.fit(x_train,epochs=50,batch_size=512,callbacks=[early_stopping])

Epoch 1/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 170s 2s/step - kl_loss: 6.7821e-04 - loss: 0.6872 - reconstruction_loss: 0.6866
Epoch 2/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - kl_loss: 0.0080 - loss: 0.6278 - reconstruction_loss: 0.6198
Epoch 3/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - kl_loss: 0.0107 - loss: 0.6185 - reconstruction_loss: 0.6077
Epoch 4/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - kl_loss: 0.0124 - loss: 0.6145 - reconstruction_loss: 0.6021
Epoch 5/50
 8/98 ━━━━━━━━━━━━━━━━━━━━ 2:26 2s/step - kl_loss: 0.0131 - loss: 0.6114 - reconstruction_loss: 0.5983

In [ ]:
def generate_images_from_latent_vectors(vae, num_images_to_generate):
  random_latent_vectors = np.random.normal(size=(num_images_to_generate, latent_dim))
  generated_images = vae.decoder.predict(random_latent_vectors)
  return generated_images
generated_images = generate_images_from_latent_vectors(vae, 20)

In [ ]:
def plot_real_and_generated_images(real_images, generated_images, num_images_to_generate):
    plt.figure(figsize=(20,10))

    for i in range(num_images_to_generate):
        plt.subplot(2, num_images_to_generate, i + 1)
        plt.imshow(real_images[i])
        plt.title('real image')
        plt.axis('off')

        plt.subplot(2, num_images_to_generate, i + 1 + num_images_to_generate)
        plt.imshow(generated_images[i])        plt.title('generated image')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

plot_real_and_generated_images(x_test, generated_images, 5)

In [ ]:
kl_loss = history.history['kl_loss']
reconstruction_loss = history.history['reconstruction_loss']
total_loss = history.history['loss']
